In [1]:
import numpy as np
from scipy.stats import truncnorm
import gym
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import collections
import matplotlib.pyplot as plt


class CEM:
    def __init__(self, n_sequence, elite_ratio, fake_env, upper_bound,
                 lower_bound):
        self.n_sequence = n_sequence
        self.elite_ratio = elite_ratio
        self.upper_bound = upper_bound
        self.lower_bound = lower_bound
        self.fake_env = fake_env

    def optimize(self, state, init_mean, init_var):
        mean, var = init_mean, init_var
        # 生成一个截断正态分布的对象
        X = truncnorm(-2, 2, loc=np.zeros_like(mean), scale=np.ones_like(var))# X服从均值为0，方差为1的截断正态分布，截断范围为[-2,2]，即X的取值范围为[-2,2]，超出范围的值被截断为2或-2
        state = np.tile(state, (self.n_sequence, 1))# 将state复制n_sequence份，组成一个矩阵，每一行都是state，共有n_sequence行

        for _ in range(5):
            # 计算动作序列的均值和方差
            lb_dist, ub_dist = mean - self.lower_bound, self.upper_bound - mean
            constrained_var = np.minimum(
                np.minimum(np.square(lb_dist / 2), np.square(ub_dist / 2)),
                var)
            # 生成动作序列
            action_sequences = [X.rvs() for _ in range(self.n_sequence)# rvs()函数用于生成随机数，生成n_sequence个随机数
                                ] * np.sqrt(constrained_var) + mean
            # 计算每条动作序列的累积奖励
            returns = self.fake_env.propagate(state, action_sequences)[:, 0]
            # 选取累积奖励最高的若干条动作序列
            elites = action_sequences[np.argsort(# np.argsort()函数返回的是数组值从小到大的索引值
                returns)][-int(self.elite_ratio * self.n_sequence):]
            new_mean = np.mean(elites, axis=0)
            new_var = np.var(elites, axis=0)
            # 更新动作序列分布
            mean = 0.1 * mean + 0.9 * new_mean
            var = 0.1 * var + 0.9 * new_var

        return mean

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device(
    "cpu")


class Swish(nn.Module):
    ''' Swish激活函数 '''
    def __init__(self):
        super(Swish, self).__init__()

    def forward(self, x):
        # x * torch.sigmoid(x) 等价于 x * torch.div(1, 1 + torch.exp(-x))
        return x * torch.sigmoid(x)


def init_weights(m):
    ''' 初始化模型权重 '''
    def truncated_normal_init(t, mean=0.0, std=0.01):
        # 初始化参数
        torch.nn.init.normal_(t, mean=mean, std=std)

        # 裁剪参数
        while True:
            cond = (t < mean - 2 * std) | (t > mean + 2 * std)
            if not torch.sum(cond):
                break
            t = torch.where(
                cond,
                torch.nn.init.normal_(torch.ones(t.shape, device=device),
                                      mean=mean,
                                      std=std), t)
        return t

    # 如果是全连接层
    if type(m) == nn.Linear or isinstance(m, FCLayer):
        # 初始化参数
        truncated_normal_init(m.weight, std=1 / (2 * np.sqrt(m._input_dim)))
        m.bias.data.fill_(0.0)


class FCLayer(nn.Module):
    ''' 集成之后的全连接层 '''
    def __init__(self, input_dim, output_dim, ensemble_size, activation):
        # 调用父类的初始化方法
        super(FCLayer, self).__init__()
        # 记录输入输出维度
        self._input_dim, self._output_dim = input_dim, output_dim
        # 初始化参数
        self.weight = nn.Parameter(
            torch.Tensor(ensemble_size, input_dim, output_dim).to(device))
        # 记录激活函数
        self._activation = activation
        # 初始化参数
        self.bias = nn.Parameter(
            torch.Tensor(ensemble_size, output_dim).to(device))

    def forward(self, x):
        # 计算输出
        return self._activation(
            torch.add(torch.bmm(x, self.weight), self.bias[:, None, :]))

In [3]:
class EnsembleModel(nn.Module):
    ''' 环境模型集成 '''
    def __init__(self,
                 state_dim,
                 action_dim,
                 ensemble_size=5,
                 learning_rate=1e-3):
        super(EnsembleModel, self).__init__()
        # 输出包括均值和方差,因此是状态与奖励维度之和的两倍
        self._output_dim = (state_dim + 1) * 2
        # 声明最大方差和最小方差两个参数,并将其设置为固定值
        self._max_logvar = nn.Parameter((torch.ones(
            (1, self._output_dim // 2)).float() / 2).to(device),
                                        requires_grad=False)
        self._min_logvar = nn.Parameter((-torch.ones(
            (1, self._output_dim // 2)).float() * 10).to(device),
                                        requires_grad=False)

        # 初始化环境模型中的参数
        self.layer1 = FCLayer(state_dim + action_dim, 200, ensemble_size,
                              Swish())
        self.layer2 = FCLayer(200, 200, ensemble_size, Swish())
        self.layer3 = FCLayer(200, 200, ensemble_size, Swish())
        self.layer4 = FCLayer(200, 200, ensemble_size, Swish())
        self.layer5 = FCLayer(200, self._output_dim, ensemble_size,
                              nn.Identity())
        self.apply(init_weights)  
        self.optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)

    def forward(self, x, return_log_var=False):
        ret = self.layer5(self.layer4(self.layer3(self.layer2(
            self.layer1(x)))))
        mean = ret[:, :, :self._output_dim // 2]
        # 在PETS算法中,将方差控制在最小值和最大值之间
        logvar = self._max_logvar - F.softplus(
            self._max_logvar - ret[:, :, self._output_dim // 2:])
        logvar = self._min_logvar + F.softplus(logvar - self._min_logvar)
        return mean, logvar if return_log_var else torch.exp(logvar)

    def loss(self, mean, logvar, labels, use_var_loss=True):
        # Calculate the inverse of the log variance
        inverse_var = torch.exp(-logvar)
        if use_var_loss:
            # Calculate the mean squared error loss
            mse_loss = torch.mean(torch.mean(torch.pow(mean - labels, 2) *
                                             inverse_var, dim=-1), dim=-1)
            # Calculate the loss for the log variance
            var_loss = torch.mean(torch.mean(logvar, dim=-1), dim=-1)
            # Add the mean squared error loss and log variance loss
            total_loss = torch.sum(mse_loss) + torch.sum(var_loss)
        else:
            # Calculate the mean squared error loss
            mse_loss = torch.mean(torch.pow(mean - labels, 2), dim=(1, 2))
            # Add the mean squared error loss
            total_loss = torch.sum(mse_loss)
        return total_loss, mse_loss

    def train(self, loss):
        self.optimizer.zero_grad()  # Zero gradients for next step.
        loss += 0.01 * torch.sum(self._max_logvar) - 0.01 * torch.sum(
            self._min_logvar)  # Add the KL divergence.
        loss.backward()  # Backpropagate the loss.
        self.optimizer.step()  # Update the parameters.


In [4]:
class EnsembleDynamicsModel:
    ''' 环境模型集成,加入精细化的训练 '''
    def __init__(self, state_dim, action_dim, num_network=5):
        self._num_network = num_network
        self._state_dim, self._action_dim = state_dim, action_dim
        self.model = EnsembleModel(state_dim,
                                   action_dim,
                                   ensemble_size=num_network)
        self._epoch_since_last_update = 0

    def train(self,
              inputs,
              labels,
              batch_size=64,
              holdout_ratio=0.1,
              max_iter=20):
        # 设置训练集与验证集
        permutation = np.random.permutation(inputs.shape[0])
        inputs, labels = inputs[permutation], labels[permutation]
        num_holdout = int(inputs.shape[0] * holdout_ratio)
        train_inputs, train_labels = inputs[num_holdout:], labels[num_holdout:]
        holdout_inputs, holdout_labels = inputs[:num_holdout], labels[:num_holdout]
        holdout_inputs = torch.from_numpy(holdout_inputs).float().to(device) # 将验证集数据转换为张量
        holdout_labels = torch.from_numpy(holdout_labels).float().to(device) # 将验证集数据转换为张量
        holdout_inputs = holdout_inputs[None, :, :].repeat( # 将验证集数据复制成网络数量的倍数
            [self._num_network, 1, 1])
        holdout_labels = holdout_labels[None, :, :].repeat( # 将验证集数据复制成网络数量的倍数
            [self._num_network, 1, 1])

        # 保留最好的结果
        self._snapshots = {i: (None, 1e10) for i in range(self._num_network)}

        for epoch in itertools.count():  # 迭代次数
            # 定义每一个网络的训练数据
            train_index = np.vstack([
                np.random.permutation(train_inputs.shape[0])
                for _ in range(self._num_network)
            ])
            # 所有真实数据都用来训练
            for batch_start_pos in range(0, train_inputs.shape[0], batch_size):
                batch_index = train_index[:, batch_start_pos:batch_start_pos +
                                          batch_size]
                train_input = torch.from_numpy(
                    train_inputs[batch_index]).float().to(device)
                train_label = torch.from_numpy(
                    train_labels[batch_index]).float().to(device)

                mean, logvar = self.model(train_input, return_log_var=True)
                loss, _ = self.model.loss(mean, logvar, train_label)
                self.model.train(loss)

            with torch.no_grad():
                mean, logvar = self.model(holdout_inputs, return_log_var=True)
                _, holdout_losses = self.model.loss(mean,
                                                    logvar,
                                                    holdout_labels,
                                                    use_var_loss=False)
                holdout_losses = holdout_losses.cpu()
                break_condition = self._save_best(epoch, holdout_losses)
                if break_condition or epoch > max_iter:  # 结束训练
                    break

    def _save_best(self, epoch, losses, threshold=0.1):
        # 1. Set updated to false
        updated = False
        # 2. Loop through each loss in the losses list
        for i in range(len(losses)):
            # 3. Get the current loss and the best loss so far
            current = losses[i]
            _, best = self._snapshots[i]
            # 4. Calculate the improvement
            improvement = (best - current) / best
            # 5. Check if the improvement is greater than the threshold
            if improvement > threshold:
                # 6. Store the current epoch and loss as the best
                self._snapshots[i] = (epoch, current)
                # 7. Set updated to true
                updated = True
        # 8. If the model has not been updated for 5 epochs, update the epoch counter
        self._epoch_since_last_update = 0 if updated else self._epoch_since_last_update + 1
        # 9. Return the epoch counter
        return self._epoch_since_last_update > 5

    def predict(self, inputs, batch_size=64):
        # A list of mean predictions.
        mean = []
        # A list of variance predictions.
        var = []
        # Iterate over the inputs in batches.
        for i in range(0, inputs.shape[0], batch_size):
            # Prepare the input.
            input = torch.from_numpy(
                inputs[i:min(i +
                             batch_size, inputs.shape[0])]).float().to(device)
            # Get the mean and variance predictions.
            cur_mean, cur_var = self.model(input[None, :, :].repeat(
                [self._num_network, 1, 1]),
                                           return_log_var=False)
            # Append the predictions to the list.
            mean.append(cur_mean.detach().cpu().numpy())
            var.append(cur_var.detach().cpu().numpy())
        # Concatenate the predictions.
        return np.hstack(mean), np.hstack(var)

In [5]:
class FakeEnv:
    def __init__(self, model):
        self.model = model

    def step(self, obs, act):
        # Model predicts the next observation and reward.
        inputs = np.concatenate((obs, act), axis=-1)
        ensemble_model_means, ensemble_model_vars = self.model.predict(inputs)
        ensemble_model_means[:, :, 1:] += obs.numpy()
        ensemble_model_stds = np.sqrt(ensemble_model_vars)

        # Sample next observation from the predicted distribution.
        ensemble_samples = ensemble_model_means + np.random.normal(
            size=ensemble_model_means.shape) * ensemble_model_stds

        # Select a model from the ensemble to use for each batch element.
        num_models, batch_size, _ = ensemble_model_means.shape
        models_to_use = np.random.choice(
            [i for i in range(self.model._num_network)], size=batch_size)
        batch_inds = np.arange(0, batch_size)
        samples = ensemble_samples[models_to_use, batch_inds]

        # Split the samples into reward and next observation.
        rewards, next_obs = samples[:, :1], samples[:, 1:]
        return rewards, next_obs

    def propagate(self, obs, actions):
        with torch.no_grad():
            # Initialize the environment.
            obs = np.copy(obs)
            total_reward = np.expand_dims(np.zeros(obs.shape[0]), axis=-1)
            obs, actions = torch.as_tensor(obs), torch.as_tensor(actions)

            # Propagate the environment forward one step at a time.
            for i in range(actions.shape[1]):
                action = torch.unsqueeze(actions[:, i], 1)
                rewards, next_obs = self.step(obs, action)
                total_reward += rewards
                obs = torch.as_tensor(next_obs)
            return total_reward

In [6]:
class ReplayBuffer:
    def __init__(self, capacity):
        # Initialize a buffer with a maximum of capacity
        self.buffer = collections.deque(maxlen=capacity)

    def add(self, state, action, reward, next_state, done):
        # Add a transition to the buffer
        self.buffer.append((state, action, reward, next_state, done))

    def size(self):
        # Return the size of the buffer
        return len(self.buffer)

    def return_all_samples(self):
        # Return all the transitions in the buffer
        all_transitions = list(self.buffer)
        state, action, reward, next_state, done = zip(*all_transitions)
        return np.array(state), action, reward, np.array(next_state), done

In [7]:
class PETS:
    ''' PETS algorithm '''
    def __init__(self, env, replay_buffer, n_sequence, elite_ratio,
                 plan_horizon, num_episodes):
        # Initialize the environment, replay buffer, and model
        self._env = env
        self._env_pool = replay_buffer

        # Get the observation space and action space dimensions
        obs_dim = env.observation_space.shape[0]
        self._action_dim = env.action_space.shape[0]
        self._model = EnsembleDynamicsModel(obs_dim, self._action_dim)
        self._fake_env = FakeEnv(self._model)
        self.upper_bound = env.action_space.high[0]
        self.lower_bound = env.action_space.low[0]

        # Initialize the CEM planner
        self._cem = CEM(n_sequence, elite_ratio, self._fake_env,
                        self.upper_bound, self.lower_bound)

        # Set the plan horizon
        self.plan_horizon = plan_horizon
        # Set the number of episodes
        self.num_episodes = num_episodes

    # Training the model
    def train_model(self):
        # Read all samples from the experience pool
        env_samples = self._env_pool.return_all_samples()
        obs = env_samples[0]
        actions = np.array(env_samples[1])
        rewards = np.array(env_samples[2]).reshape(-1, 1)
        next_obs = env_samples[3]
        inputs = np.concatenate((obs, actions), axis=-1)
        labels = np.concatenate((rewards, next_obs - obs), axis=-1)
        # Train the model using the samples in the experience pool
        self._model.train(inputs, labels)

    def mpc(self):
        # Initialize the mean and variance of the action distribution
        mean = np.tile((self.upper_bound + self.lower_bound) / 2.0,
                       self.plan_horizon)
        var = np.tile(
            np.square(self.upper_bound - self.lower_bound) / 16,
            self.plan_horizon)
        # Reset the environment, and start a new episode
        obs, done, episode_return = self._env.reset(), False, 0
        while not done:
            # Use CEM to optimize the mean and variance of the action distribution
            actions = self._cem.optimize(obs, mean, var)
            # Select the first action
            action = actions[:self._action_dim]  
            # Take the action and get the next observation, reward, and done flag
            next_obs, reward, done, _ = self._env.step(action)
            # Add the transition to the environment pool
            self._env_pool.add(obs, action, reward, next_obs, done)
            # Update the observation
            obs = next_obs
            # Update the episode return
            episode_return += reward
            # Update the mean and variance of the action distribution
            mean = np.concatenate([
                np.copy(actions)[self._action_dim:],
                np.zeros(self._action_dim)
            ])
        return episode_return

    def explore(self):
        obs, done, episode_return = self._env.reset(), False, 0
        while not done:
            # Sample an action from the environment's action space
            action = self._env.action_space.sample()
            # Take the action in the environment
            next_obs, reward, done, _ = self._env.step(action)
            # Add the new experience to the environment pool
            self._env_pool.add(obs, action, reward, next_obs, done)
            # Set the next observation as the current observation
            obs = next_obs
            # Add the reward to the episode return
            episode_return += reward
        return episode_return

    def train(self):
        # 初始化返回值列表
        return_list = []
        # 随机探索一条序列，探索的目的是收集数据
        explore_return = self.explore()
        # 打印当前随机策略的得分
        print('episode: 1, return: %d' % explore_return)
        # 将得分加入到返回值列表中
        return_list.append(explore_return)

        # 进行多次策略更新和模型更新
        for i_episode in range(self.num_episodes - 1):
            # 进行策略更新
            self.train_model()
            # 进行模型更新
            episode_return = self.mpc()
            # 将得分加入到返回值列表中
            return_list.append(episode_return)
            # 打印当前策略的得分
            print('episode: %d, return: %d' % (i_episode + 2, episode_return))
        # 返回返回值列表
        return return_list

In [8]:
buffer_size = 100000
n_sequence = 50
elite_ratio = 0.2
plan_horizon = 25
num_episodes = 10
env_name = 'Pendulum-v0'
env = gym.make(env_name)

replay_buffer = ReplayBuffer(buffer_size)
pets = PETS(env, replay_buffer, n_sequence, elite_ratio, plan_horizon,
            num_episodes)
return_list = pets.train()

episodes_list = list(range(len(return_list)))
plt.plot(episodes_list, return_list)
plt.xlabel('Episodes')
plt.ylabel('Returns')
plt.title('PETS on {}'.format(env_name))
plt.show()

c:\vnstudio\lib\site-packages\gym\envs\registration.py:556: UserWarning: WARN: The environment Pendulum-v0 is out of date. You should consider upgrading to version `v1`.
  f"The environment {id} is out of date. You should consider "


DeprecatedEnv: Environment version v0 for `Pendulum` is deprecated. Please use `Pendulum-v1` instead.